## Fine-Tuning, Guidance and Conditioning

## 1、微调：
    使用新数据重新训练现有模型，以改变它们产生的输出类型
    新数据与基础模型的原始训练数据有些相似（例如，如果您尝试生成卡通人脸，从人脸训练模型开始可能是个好主意），则微调通常效果最佳，但令人惊讶的是，即使领域发生很大变化，其好处仍然存在。
## 2、指导
    采用现有模型并在推理时引导生成过程以获得额外的控制
    训练一个条件模型（下一节将详细介绍），该模型接受其他输入来帮助引导生成过程
    如果我们已经有一个想要使用的训练有素的无条件模型，该怎么办？
    输入指导，即根据某些指导函数评估生成过程中每一步的模型预测并进行修改，以使最终生成的图像更符合我们的喜好。  
（名为 CLIP 的强大预训练模型的示例，该模型让我们能够根据文本描述指导生成。）
## 3、调节（条件）
    创建了一个条件模型，我们可以在推理时通过控制输入的内容来控制它
    （本实例中该模型学习根据类标签生成图像）
## 有多种方法可以传递这些条件信息：
    将其作为 UNet 输入中的附加通道馈入。这通常在条件信息与图像形状相同时使用，例如分割蒙版、深度图或图像的模糊版本（在恢复/超分辨率模型的情况下）。它也适用于其他类型的条件。例如，在笔记本中，类标签被映射到嵌入，然后扩展为与输入图像相同的宽度和高度，以便可以将其作为附加通道馈入。    
创建嵌入，然后将其投影到与 UNet 一个或多个内部层的输出通道数相匹配的大小，然后将其添加到这些输出。例如，这就是处理时间步长条件的方式。每个 Resnet 块的输出都添加了投影的时间步长嵌入。当您将 CLIP 图像嵌入等向量作为条件信息时，这很有用。一个值得注意的例子是稳定扩散的“图像变化”版本，它正是这样做的。    
添加交叉注意层，可以“关注”作为条件传入的序列。当条件以文本形式呈现时，这最有用 - 使用转换器模型将文本映射到嵌入序列，然后使用 UNet 中的交叉注意层将此信息合并到去噪路径中。我们将在第 3 单元中看到这一点，我们将研究稳定扩散如何处理文本条件。

## 读完本笔记本后，你将了解如何：
    
使用新的调度程序创建采样循环并更快地生成样    本
根据新数据对现有的扩散模型进行
    微调，包括：
使用梯度积累来解决小批量的一些问题
将样本记录到权重和偏差中以监控进度（通过附带的示例脚本）
保存生成的管道并将其    上传到中心
使用额外的损失函数指导采样过程，以增加对现有模型的        控制，包括：
使用简单的基于颜色的损失探索        不同的指导方法
使用 CLIP 通过        文本提示指导生成
使用 Gradio 和 🤗 Spaces 共享自定义采样循环

In [3]:
import numpy as np
import torch
import torch.nn.functional as F
import torchvision
from datasets import load_dataset
from diffusers import DDIMScheduler, DDPMPipeline
from matplotlib import pyplot as plt
from PIL import Image
from torchvision import transforms
from tqdm.auto import tqdm

AttributeError: partially initialized module 'torch' has no attribute 'fx' (most likely due to a circular import)

CLIP指导：